import pandas as pd
import numpy as np
import datetime 
import tqdm

import requests
import json

from string import punctuation
from nltk.corpus import stopwords
from gensim.models.keyedvectors import KeyedVectors
from nltk.tokenize import WhitespaceTokenizer
exclude = set(punctuation + u'0123456789[]—«»–')
from pymorphy2 import MorphAnalyzer

from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import word2vec

from catboost import Pool, CatBoostClassifier
from  sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

from itertools import product
from __future__ import division
from collections import Counter
import functools
import re
import string
import nltk

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ndmel\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
pd.set_option('max_colwidth', 1000)

In [4]:
remove_punctuation_map = dict((ord(char), None) for char in string.punctuation)
stemmer = nltk.stem.porter.PorterStemmer()

def stem_tokens(tokens):
    return [stemmer.stem(item) for item in tokens]

def preprocess(text):
    return ' '.join(stem_tokens(nltk.word_tokenize(text.lower().translate(remove_punctuation_map))))

# def preprocess(text):
#     buf = ''.join(ch for ch in text if ch not in exclude)
#     tokens = WhitespaceTokenizer().tokenize(buf.lower())
#     lemmatizer = MorphAnalyzer()
#     lemmas = []
#     for t in tokens[:]:
#         if not t in stopwords.words('russian'):
#             try:
#                 lemma = lemmatizer.parse(t)[0].normal_form
#             except: 
#                 lemma = t
#             lemmas.append(lemma)
#     return ' '.join(lemmas)

In [5]:
@functools.lru_cache(maxsize=2 ** 19)
def uniq_words(text):
    return set(re.findall("\w+", text))

def calculate_idfs(data):
    counter = Counter()
    uniq_skills = list()
    for skill_set in tqdm.tqdm(data['skill_set_preprocessed'], desc="calc idf for paragraph"):
        counter.update(set(skill_set))
        
    num_docs = data.shape[0]
    idfs = {}
    for word in counter:
        idfs[word] = counter[word]
    return idfs

In [6]:
def texts_overlap(t1, t2):
    
    '''
        t1 - skill
        t2 - search text
    '''
    
    if pd.isnull(t1) or len(t1) == 0:
        return 0
    if pd.isnull(t2) or len(t2) == 0:
        return 0
    
    word_count = 0
    for word1 in t1.split(' '):
        if word1 in t2.split(' '):
            word_count += 1
            
    return word_count / len(t1.split(' '))

In [7]:
def fill_resume(id_url):
    
    count = resume_df.shape[0]
    r = requests.get(base_url + id_url)
    resume = json.loads(r.text)
    
    # base
    resume_df.loc[count, 'id'] = resume.get('id')
#     resume_df.loc[count, 'age'] = resume.get('age')
#     resume_df.loc[count, 'first_name'] = resume.get('first_name')
#     resume_df.loc[count, 'last_name'] = resume.get('last_name')
    resume_df.loc[count, 'skill_set'] = str(resume.get('skill_set'))
    resume_df.loc[count, 'skill_set_preprocessed'] = str([preprocess(x) for x in resume.get('skill_set')])

    # jobs
#     count_jobs = jobs_df.shape[0]
#     if not pd.isnull(resume.get('experience')).all():
#         for e, experience in enumerate(resume.get('experience')):

#             jobs_df.loc[count_jobs + e, 'resume_id'] = resume.get('id')
#             jobs_df.loc[count_jobs + e, 'company_id'] = experience.get('company_id')
#             jobs_df.loc[count_jobs + e, 'company'] = experience.get('company')
#             jobs_df.loc[count_jobs + e, 'description'] = experience.get('description')
#             jobs_df.loc[count_jobs + e, 'start'] = experience.get('start')
#             jobs_df.loc[count_jobs + e, 'end'] = experience.get('end')
#             jobs_df.loc[count_jobs + e, 'position'] = experience.get('position')
            
#             # area
#             area = experience.get('area')
#             if not pd.isnull(area):
#                 jobs_df.loc[count_jobs + e, 'area_id'] = area.get('id')
#                 jobs_df.loc[count_jobs + e, 'area'] = area.get('name') 

## данные

In [8]:
chat_ids = ['2f8f6ec6000435cfc70039ed1f45306e387558', '016bea2e0000c902ea0039ed1f514235754343', 'cd9c1e66000109eb5c0039ed1f6d726b6a4937', '6b8c829b00038ee9470039ed1f6c4a30326a43','337f25cc000380a8ca0039ed1f7745554c5333',
           '13855d8f000403568f0039ed1f796833587a4f', 'ef2b55ce0001ef0f400039ed1f4c73726d6464', '22bd974a00036f11c80039ed1f65375736594e', 'a5d68189000274614d0039ed1f6262706b3948']
medic_ids = ['1dbe69160001ae8a390039ed1f364134526d46','a81f07b600013c595f0039ed1f695533515773', 'd310507e0001471d4d0039ed1f47674c536768', '922449e2000130dc850039ed1f786e685a5732', 'c8c62e31000189fc690039ed1f62354932386b',
            '883f2c570002db914f0039ed1f56556561587a', 'b91eae8d0003e9b3510039ed1f396646413130', '608f15ac00031bacc90039ed1f33465375624c', 'e84d0d950003c0cad70039ed1f394f65473937', 'a6238e73000354ac6e0039ed1f663970634966']
driver_ids = ['6415d7eb00007a36710039ed1f736563726574', '846da09a00021b77e20039ed1f63576b6f3832', '6c38ef930003d1eae30039ed1f365043694b35', 'a655dc4900042abd5e0039ed1f59707647684e', 'f82208070002ca2a740039ed1f4f4c4d716e30',
             '73c897850000930ecb0039ed1f736563726574', 'f4d1f06f00033573de0039ed1f51674b745357', '5ecd54e000023bd7970039ed1f796f66497a68', '907bb8410004350ce50039ed1f324c44303745']

analytics = ['5141528400020285a80039ed1f6d434b776667', 'bd94952e0003723cf80039ed1f7448774f6d69', '3446b6820003c3384d0039ed1f4e4a72776966', '06a56f1e000404c1ae0039ed1f546f74437a7a',
             '90733f8500041d0e4e0039ed1f51324649766a', 'a0f42b320000e01d580039ed1f774756544f42', '2425ef0c00041dfaef0039ed1f6977786a6442', 'a61ac917000127e38d0039ed1f31716162796d', '05bc19c4000206b5c40039ed1f555058764c34',
             '4471c6c800004fdf670039ed1f736563726574', '6578189d0000ad2f710039ed1f736563726574', 'dad443cb000247d5c10039ed1f78554b675143', 'fd2167820003f074510039ed1f61536e433246', '1d79352100029e53930039ed1f417a75397773', 
             'f0a03b4b0002d893070039ed1f54304c4f7872', 'fd2bb2130001fb43e80039ed1f305a5a333166', '9abe579e0000bafce70039ed1f736563726574', 'c21ddb3c00025400940039ed1f596743595774', 'a97d01350003092e090039ed1f6465694e7661',
             'fba5505c0002ca04b60039ed1f555856796b55']

all_ids = np.append(chat_ids, np.append(medic_ids, np.append(analytics, driver_ids)))

base_url = 'https://api.hh.ru/resumes/'

# jobs_df = pd.DataFrame(columns = ['resume_id', 'company_id', 'company', 'start', 'end', 'description', 'area', 'area_id', 'position'])
resume_df = pd.DataFrame(columns = ['id', 'skill_set', 'skill_set_preprocessed'])

In [9]:
for id_url in tqdm.tqdm(all_ids, total = len(all_ids)):
    fill_resume(id_url)

100%|██████████████████████████████████████████████████████████████████████████████████| 48/48 [00:10<00:00,  4.51it/s]


In [10]:
resume_df.to_csv('data/resume_df_1.csv', sep=';', encoding='cp1251', index = False)

In [227]:
resume_df = pd.read_csv('data/resume_df_1.csv', sep=';', encoding='cp1251')
resume_df.skill_set = resume_df.skill_set.map(lambda x: x.strip("[]").replace('\'', '').split(', '))
resume_df.skill_set_preprocessed = resume_df.skill_set_preprocessed.map(lambda x: x.strip("[]").replace('\'', '').split(', '))

## Описание вакансии

In [229]:
import pickle

In [230]:
def save_obj(obj, name ):
    with open('data/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open('data/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [245]:
resume_df.skill_set.values

array([ list(['Python', 'Celery', 'Flask', 'Redis', 'RabbitMQ', 'PostgreSQL', 'Docker', 'Docker Swarm', 'Sentry', 'Linux', 'Reverse engineering', 'React', 'ES6', 'Git', 'WebSocket']),
       list(['Управление лояльностью потребителей', 'Оптимизация бизнес-процессов', 'Руководство коллективом', 'CRM', 'Развитие дистанционных каналов продаж и обслуживания', 'Продажи через интернет', 'Прямые продажи']),
       list(['']),
       list(['Internet Marketing', 'Яндекс.Метрика', 'Google Analytics', 'Google AdWords', 'Яндекс.Директ', 'Контекстная реклама', 'CRM', 'Управление отношениями с клиентами', 'Интернет-реклама', 'Баннерная реклама', 'Yandex Direct', 'Лидогенерация', 'E-Mail Marketing', 'E-Commerce', 'Social Media Marketing', 'B2C маркетинг', 'B2B Marketing', 'Интернет маркетинг', 'Маркетинг услуг', 'Информационные технологии', 'Usability', 'WEB аналитика', 'ERP Systems', 'RTB', 'CPA', 'Аналитика продаж', 'Автоматизация продаж', 'Автоматизация контекстных кампаний', 'SEM', 'SEO оптимизац

In [246]:
tests = ['Хотим найти аналитик со занием ms word, ms excel, ms powerpoint.\
В его задачи будет входить оптимизация бизнес-процессов, руководство стажерами по направлению, web аналитика, аналитика продаж, автоматизация продаж.\
Плюсом будет умение подготовки и организации презентаций, знание google analytics и google AdWords.']

In [247]:
idfs = calculate_idfs(resume_df)
idfs_mean = np.mean(list(idfs.values()))

save_obj(idfs, 'idfs')

calc idf for paragraph: 100%|██████████████████████████████████████████████████████████████████| 48/48 [00:00<?, ?it/s]


In [248]:
test_df = pd.DataFrame()

count = 0
for test in tqdm.tqdm(tests, total = len(tests)):
    test_df.loc[count, 'text'] = test
    test = preprocess(test)
    
    skills = []
    for index, row in resume_df.iterrows():
        skill_set = row.skill_set_preprocessed
        
        for skill in skill_set:
            if skill in test:
                skills.append(skill)
            elif texts_overlap(skill, test) >= 0.5:
                skills.append(skill)
    
    if len(skills) != 0:
        skills = set(skills)
        skills = [x for x in skills if len(x) > 0]
        test_df.loc[count, 'skills'] = str(list(skills))
        test_df.loc[count, 'essential_skills'] = str([x for x in skills if idfs.get(x) > idfs_mean])
        test_df.loc[count, 'nice_to_have_skills'] = str([x for x in skills if idfs.get(x) <= idfs_mean])

    count += 1
    

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 62.32it/s]


In [249]:
test_df

,text,skills,essential_skills,nice_to_have_skills
0,"Хотим найти аналитик со занием ms word, ms excel, ms powerpoint, трудолюбивого, имеющего водительское удостоверение, пунктуального.В его задачи будет входить оптимизация бизнес-процессов, руководство стажерами по направлению, web аналитика, аналитика продаж, автоматизация продаж.Плюсом будет умение подготовки и организации презентаций, знание google analytics и google AdWords.","['водительское удостоверение категории bc', 'adob analyt', 'ms powerpoint', 'автоматизация продаж', 'ms visio', 'аналитика', 'seo оптимизация', 'водительское удостоверение категории b', 'ms outlook', 'googl doc', 'руководство коллективом', 'googl adword', 'ms offic', 'прогнозирование продаж', 'организация презентаций', 'c', 'ms sql', 'ms word', 'ms project', 'erp', 'ms excel', 'проведение презентаций', 'оптимизация бизнеспроцессов', 'подготовка презентаций', 'web аналитика', 'ms access', 'googl analyt', 'оптимизация налогообложения', 'аналитика продаж']","['ms powerpoint', 'аналитика', 'водительское удостоверение категории b', 'ms outlook', 'руководство коллективом', 'googl adword', 'ms offic', 'c', 'ms sql', 'ms word', 'ms excel', 'проведение презентаций', 'подготовка презентаций', 'web аналитика', 'googl analyt']","['водительское удостоверение категории bc', 'adob analyt', 'автоматизация продаж', 'ms visio', 'seo оптимизация', 'googl doc', 'прогнозирование продаж', 'организация презентаций', 'ms project', 'erp', 'оптимизация бизнеспроцессов', 'ms access', 'оптимизация налогообложения', 'аналитика продаж']"


## Достаем резюме по описанию вакансии

In [250]:
test_df.skills = test_df.skills.map(lambda x: x.strip("[]").replace('\'', '').split(', '))
test_df.essential_skills = test_df.essential_skills.map(lambda x: x.strip("[]").replace('\'', '').split(', '))
test_df.nice_to_have_skills = test_df.nice_to_have_skills.map(lambda x: x.strip("[]").replace('\'', '').split(', '))

In [251]:
result = pd.DataFrame(columns=np.append(test_df.columns, np.append(resume_df.columns, ['es_skills_counter', 'nth_skills_counter'])))

count = 0
for index, test in tqdm.tqdm(test_df.iterrows(), total = test_df.shape[0]):

    for jindex, row in resume_df.iterrows():

        es_skills = test.essential_skills
        es_skills_counter = 0

        nth_skills = test.nice_to_have_skills
        nth_skills_counter = 0

        for skill in row.skill_set_preprocessed:
#             print(es_skills)
            if pd.notnull(es_skills).all() and skill in es_skills:
                es_skills_counter += 1

            if pd.notnull(nth_skills).all() and skill in nth_skills:
                nth_skills_counter += 1

        if es_skills_counter > 0 or nth_skills_counter > 0:
            result.loc[count, test_df.columns.values] = test
            result.loc[count, resume_df.columns.values] = row
            result.loc[count, 'es_skills_counter'] = es_skills_counter
            result.loc[count, 'nth_skills_counter'] = nth_skills_counter
            
            count += 1

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 15.34it/s]


In [252]:
result.sort_values(['text', 'es_skills_counter', 'nth_skills_counter'], ascending=False)[['text', 'essential_skills', 'nice_to_have_skills', 'id', 'skill_set', 'es_skills_counter', 'nth_skills_counter']]

,text,essential_skills,nice_to_have_skills,id,skill_set,es_skills_counter,nth_skills_counter
11,"Хотим найти аналитик со занием ms word, ms excel, ms powerpoint, трудолюбивого, имеющего водительское удостоверение, пунктуального.В его задачи будет входить оптимизация бизнес-процессов, руководство стажерами по направлению, web аналитика, аналитика продаж, автоматизация продаж.Плюсом будет умение подготовки и организации презентаций, знание google analytics и google AdWords.","[ms powerpoint, аналитика, водительское удостоверение категории b, ms outlook, руководство коллективом, googl adword, ms offic, c, ms sql, ms word, ms excel, проведение презентаций, подготовка презентаций, web аналитика, googl analyt]","[водительское удостоверение категории bc, adob analyt, автоматизация продаж, ms visio, seo оптимизация, googl doc, прогнозирование продаж, организация презентаций, ms project, erp, оптимизация бизнеспроцессов, ms access, оптимизация налогообложения, аналитика продаж]",2425ef0c00041dfaef0039ed1f6977786a6442,"[Английский язык, MS Office, MS PowerPoint, SMM, MS Excel, ERP, 1С, WEB аналитика, Веб-аналитика, Google Analytics, Яндекс.Метрика, MS SQL, Аналитика, Python]",7,1
17,"Хотим найти аналитик со занием ms word, ms excel, ms powerpoint, трудолюбивого, имеющего водительское удостоверение, пунктуального.В его задачи будет входить оптимизация бизнес-процессов, руководство стажерами по направлению, web аналитика, аналитика продаж, автоматизация продаж.Плюсом будет умение подготовки и организации презентаций, знание google analytics и google AdWords.","[ms powerpoint, аналитика, водительское удостоверение категории b, ms outlook, руководство коллективом, googl adword, ms offic, c, ms sql, ms word, ms excel, проведение презентаций, подготовка презентаций, web аналитика, googl analyt]","[водительское удостоверение категории bc, adob analyt, автоматизация продаж, ms visio, seo оптимизация, googl doc, прогнозирование продаж, организация презентаций, ms project, erp, оптимизация бизнеспроцессов, ms access, оптимизация налогообложения, аналитика продаж]",a97d01350003092e090039ed1f6465694e7661,"[MS Excel, Подготовка отчетов, MS Word, MS PowerPoint, Яндекс.Директ, Яндекс.Метрика, Google AdWords, Google Analytics, Копирайтинг, EMAIL, HTML, JOOMLA, WIX, Создание сайтов, Работа с доменами, Таргетинг, SMM, Adobe Photoshop, Английский язык, Google Docs, Adobe Analytics, Social Media Marketing, Keynotes, Ведение групп в социальных сетях, MS Project, SERM, CPC, CPM, CPL, UTM]",5,3
2,"Хотим найти аналитик со занием ms word, ms excel, ms powerpoint, трудолюбивого, имеющего водительское удостоверение, пунктуального.В его задачи будет входить оптимизация бизнес-процессов, руководство стажерами по направлению, web аналитика, аналитика продаж, автоматизация продаж.Плюсом будет умение подготовки и организации презентаций, знание google analytics и google AdWords.","[ms powerpoint, аналитика, водительское удостоверение категории b, ms outlook, руководство коллективом, googl adword, ms offic, c, ms sql, ms word, ms excel, проведение презентаций, подготовка презентаций, web аналитика, googl analyt]","[водительское удостоверение категории bc, adob analyt, автоматизация продаж, ms visio, seo оптимизация, googl doc, прогнозирование продаж, организация презентаций, ms project, erp, оптимизация бизнеспроцессов, ms access, оптимизация налогообложения, аналитика продаж]",337f25cc000380a8ca0039ed1f7745554c5333,"[MS Word, MS Excel, MS PowerPoint, Prezi, MS SQL, Access, Mathcad, MATLAB, C/C++, Python, Английский язык, AutoCAD, Windows Forms, Qt, C#, ARIS, Pega, Atlassian Jira, Oxygen Xml Editor, Unix, SOAP]",5,0
10,"Хотим найти аналитик со занием ms word, ms excel, ms powerpoint, трудолюбивого, имеющего водительское удостоверение, пунктуального.В его задачи будет входить оптимизация бизнес-процессов, руководство стажерами по направлению, web аналитика, аналитика продаж, автоматизация продаж.Плюсом будет умение подготовки и организации презентаций, знание google ana

## result

In [206]:
j_str = "{"

j_str = j_str + """"es_skills":{0},
               "nth_skills":["GG", "GG"],
               "resume":[""".format(str([x for x in resume_df.skill_set.values[0]]).replace("'", "\""))

for index, row in resume_df.head().iterrows():
    id = row.id
    es_skill_p = 0.3
    nth_skill_p = 0.5
    
    j_str = j_str + "{"
    j_str = j_str + """"id":{0}, "es_skills_p":{1}, "nth_skills":{2}""".format("\"eflkej\"", 55, 65)
    j_str = j_str + "},"

j_str = j_str.rstrip(',') + ']}'

j = json.loads(j_str)

j

{'es_skills': ['Python',
  'Celery',
  'Flask',
  'Redis',
  'RabbitMQ',
  'PostgreSQL',
  'Docker',
  'Docker Swarm',
  'Sentry',
  'Linux',
  'Reverse engineering',
  'React',
  'ES6',
  'Git',
  'WebSocket'],
 'nth_skills': ['GG', 'GG'],
 'resume': [{'es_skills_p': 55, 'id': 'eflkej', 'nth_skills': 65},
  {'es_skills_p': 55, 'id': 'eflkej', 'nth_skills': 65},
  {'es_skills_p': 55, 'id': 'eflkej', 'nth_skills': 65},
  {'es_skills_p': 55, 'id': 'eflkej', 'nth_skills': 65},
  {'es_skills_p': 55, 'id': 'eflkej', 'nth_skills': 65}]}